# MSIA - Points - 4 - Machine learning

This practical session is our first machine learning session.
We target semantic segmentation for outdoor scenes.
We will compute local descriptors in a point cloud and use these descriptors to estimate the point labels.

We will roughly follow the procedure described in *Semantic Classification of 3D Point Clouds with Multiscale Spherical Neighborhoods*, Thomas et al.

The point clouds are sub point clouds from [NPM3D](https://npm3d.fr/), a benchmark suite for lidar semantic segmentation.
If interested, please login to the benchmark and test your ideas.

## Notebook setup

For point cloud loading we use here `plyfile` library, that can load a point cloud in ply format with its attributes (here a class label).

Then, we download the point cloud we will work with. The point cloud can be directly donwloaded from the course [github]( https://github.com/aboulch/MSIA_points) or using the following command.

In [ ]:
# install the missing package
!pip install plyfile

# load the point clouds collection of point clouds
!wget https://github.com/aboulch/MSIA_points/releases/download/v0.0.0/Lille1_1_sub_train.ply
!wget https://github.com/aboulch/MSIA_points/releases/download/v0.0.0/Lille1_1_sub_val.ply

Finally, we import the libraries useful for this practical session.
The whole practical session will be managed with numpy.

In [ ]:
import numpy as np
import tqdm
import plotly.graph_objects as go # for visualization
from scipy.spatial import KDTree
from plyfile import PlyData
from scipy.spatial import KDTree
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics as sk_metrics

global_render_mode = True # enable / disable the plotly rendering

Similarly to the first practical session, we use [plotly](https://plotly.com/python) for point cloud visualization.
Note that the function can work with both colors as a 1D array or with a 3D array.

In the later case, colors must be intergers in $[0,255]$.

In the former case, the colors are displayed using the color scale "viridis".

In [ ]:
def point_cloud_visu(pts, cls=None):

  if global_render_mode:

    fig = go.Figure(
        data=[
            go.Scatter3d(
                x=pts[:,0], y=pts[:,1], z=pts[:,2],
                mode='markers',
                marker=dict(size=3,
                            color=cls,
                            # colorscale='Viridis',
                            )
            )
        ],
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False),
                aspectmode="data", #this string can be 'data', 'cube', 'auto', 'manual'
                #a custom aspectratio is defined as follows:
                aspectratio=dict(x=1, y=1, z=0.95)
            )
        )
    )
    fig.show()

Finally, we load the point clouds (one for training, one for validation).
The points are in `pts` and `val_pts` and the class labels (intergers) are in `cls` and `val_cls`.

In [ ]:
with open("Lille1_1_sub_train.ply", 'rb') as f:
  plydata = PlyData.read(f)
  pts = np.stack([plydata['vertex']["x"],plydata['vertex']["y"], plydata['vertex']["z"]], axis=1)
  cls = plydata['vertex']['scalar_class'].astype(np.int64)

with open("Lille1_1_sub_val.ply", 'rb') as f:
  plydata = PlyData.read(f)
  val_pts = np.stack([plydata['vertex']["x"],plydata['vertex']["y"], plydata['vertex']["z"]], axis=1)
  val_cls = plydata['vertex']['scalar_class'].astype(np.int64)

class_mapping = [
  "unclassified",
  "ground",
  "building",
  "pole - road sign - traffic light",
  "bollard - small pole",
  "trash can",
  "barrier",
  "pedestrian",
  "car",
  "natural - vegetation",
]

color_map = np.array([
    [0,0,0],
    [255, 0, 255],
    [0, 200, 255],
    [150, 240, 255],
    [180, 30, 80],
    [0, 150, 255],
    [50, 120, 255],
    [30, 30, 255],
    [245, 150, 100],
    [0, 175, 0],
], dtype=np.uint8)

## Point cloud preparation

The point clouds are too dense if we want to compute in a reasonable time (for a practical session) one complex descriptor for each point and train a classifier on top of it.

As preliminary, we select a reasonable number of *support points*, the points that will be use to compute the descriptors.

**Question**: compute the new point cloud `val_pts_for_descriptor` and `val_cls_for_descriptor` by downsampling by 10 the validation point cloud (it does not need to, just taking one point every 10 is sufficient).

In [ ]:
def decimate_val_points(val_pts, val_cls):
  ## fill the function

  # To be removed (only there for compatibility)
  val_pts_for_descriptor = val_pts[:4]
  val_cls_for_descriptor = val_cls[:4]

  return val_pts_for_descriptor, val_cls_for_descriptor

val_pts_for_descriptor, val_cls_for_descriptor = decimate_val_points(val_pts, val_cls)
print(f"Validation point cloud: {val_pts.shape[0]} points")
print(f"Validation point cloud for descriptor: {val_pts_for_descriptor.shape[0]} points")
point_cloud_visu(val_pts_for_descriptor, color_map[val_cls_for_descriptor])

Downsampling the training point cloud is a bit more tricky. Indeeed, the classes are very unbalanced in a urban scene: a lot road points, not so much on cars, very little for poles and traffic signs.
Thus, training classic machine learning algorithms, like random forests, can be difficult.

**Question:** create the support points `pts_for_descriptor` and `cls_for_descriptor` in a more or less balanced fashion following the given algorithm.

Parameters: `max_pts_per_class`, is the maximal number of points for a given class, e.g., 1000.

- create lists for points and classes
- For each class $c$ (for $c$ > 0, we do not keep the unassigned points)
  - get the points corresponding to this class
  - if the number of points of this class > `max_pts_per_class`:
    - randomly select `max_pts_per_class` in these points
  - else:
    - select all the points
  - add the correspoding points to the containers
- convert lists to numpy arrays

In [ ]:
def decimate_training_points( pts, cls, max_pts_per_class = 1000):
  ## fill the function

  # To be removed (only there for compatibility)
  pts_for_descriptor = pts[:4]
  cls_for_descriptor = cls[:4]

  return pts_for_descriptor, cls_for_descriptor

pts_for_descriptor, cls_for_descriptor = decimate_training_points( pts, cls, max_pts_per_class = 1000)
print(f"Training point cloud: {pts.shape[0]} points")
print(f"Training point cloud for descriptor: {pts_for_descriptor.shape[0]} points")
point_cloud_visu(pts_for_descriptor, color_map[cls_for_descriptor])



## Creating the descriptors

In this section, we will create the descriptors associated with each point.
As described in *Thomas et al*, the descriptors rely on the covariance matrix computed on a small neighborhood around each support point.

**Question:** compute the neighborhoods of each support point from `pts_for_descriptor` (resp. `val_pts_for_descriptor`) in the orginal point cloud `pts` (resp. 'val_pts').

*Note:* you can use the [KDTree from scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.KDTree.html).

In [ ]:
def compute_radius_neighborhoods(support_points, points, radius=2):
  ## fill the function

  # To be removed (only there for compatibility)
  indices = [[0,1,2]]

  return indices

indices = compute_radius_neighborhoods(pts_for_descriptor, pts, radius=2)
val_indices = compute_radius_neighborhoods(val_pts_for_descriptor, val_pts, radius=2)

**Question:** compute the covaiance matrix, eigen values, and eigen vectors for a list of points. The function returns the eigen values and eigen vectors, sorted in **descending** order.

*Note:* you can use the numpy function for [covariance computation](https://numpy.org/doc/stable/reference/generated/numpy.cov.html) as well as the [eigen values/vector extractor](https://numpy.org/doc/stable/reference/generated/numpy.linalg.eigh.html).

*Note-2:* the `eigh` value from numpy return the values in **ascending order**.

In [ ]:
def compute_eigen(pts):
  # input: point cloud [N 3]
  # output: eigen values [3], eigen vectors [3x3]

  ## fill the function

  # To be removed (only there for compatibility)
  eigvals = np.array([1., 0.5, 0.])
  eigvecs = np.eye(3)

  return eigvals, eigvecs

**Question:** compute eigen values and eigen vectors for all the support points (training and validation) by completing the function. The function loop over all the support points and their neighborhood indices and returns the eigen values and eigen vector as numpy arrays.

In [ ]:
def compute_eigen_for_all_support_points(points, indices):
  ## fill the function

  # To be removed (only there for compatibility)
  eigvals = np.ones((5,3))
  eigvecs = np.zeros((5,3,3))

  # safeguard (all eigenvalues should >= 0) --> to be kept
  eigvals = np.clip(eigvals, a_min=0, a_max=np.inf)
  return eigvals, eigvecs

eigvals, eigvecs = compute_eigen_for_all_support_points(pts, indices) # indices has the length of the support points
val_eigvals, val_eigvecs = compute_eigen_for_all_support_points(val_pts, val_indices) # indices has the length of the support points

We now compute the descriptors per se.
In this part of the practical session, we only compute a subset of the descriptors presented in *Thomas et al.*.

In the following $\lambda_1 \geq \lambda_2 \geq \lambda_3$ are the eigen values in descending order, and $v_1, v_2, v_3$ their associated unit eigen vector.


**Question:** Linearity describe how much the neighborhood fits a line. It is defined by:
$$D_\text{linearity}(P) = 1 - \frac{\lambda_2}{\lambda_1 + \epsilon}$$
Fill the corresponding function.

*Note:* $\lambda_1$ can be $0$, $\epsilon = 10^{-12}$ is a small constant to prevent undefined values.

In [ ]:
def descriptor_linearity(eigvals):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvals.shape[0], 1)

  assert(len(desc.shape)==2) # shape should be [N,1]
  return desc

d_linearity = descriptor_linearity(eigvals)
val_d_linearity = descriptor_linearity(val_eigvals)

# display for training point cloud
point_cloud_visu(pts_for_descriptor, d_linearity[:,0])

**Question:** Planarity describe how much the neighborhood fits a plane. It is defined by:
$$D_\text{planarity}(P) = \frac{\lambda_2 - \lambda_3}{\lambda_1 + \epsilon}$$
Fill the corresponding function.

*Note:* $\lambda_1$ can be $0$, $\epsilon = 10^{-12}$ is a small constant to prevent undefined values.

In [ ]:
def descriptor_planarity(eigvals):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvals.shape[0], 1)

  assert(len(desc.shape)==2) # shape should be [N,1]
  return desc

d_planarity = descriptor_planarity(eigvals)
val_d_planarity = descriptor_planarity(val_eigvals)
point_cloud_visu(pts_for_descriptor, d_planarity[:,0])

**Question:** verticality describe how much the neighborhood structure aligns with the vertical direction, i.e., how much the normal is orthogonal to the vertical direction. It is defined by:
$$D_\text{verticality}(P) = 1 -  \left| <v_3, e_z>\right|$$
Fill the corresponding function.

In [ ]:
def descriptor_verticality(eigvecs):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvecs.shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

d_verticality = descriptor_verticality(eigvecs)
val_d_verticality = descriptor_verticality(val_eigvecs)
point_cloud_visu(pts_for_descriptor, d_verticality[:,0])

**Question:** sphericity describe how much the neighborhood fits a sphere. It is defined by:
$$D_\text{sphericity}(P) = \frac{\lambda_3}{\lambda_1 + \epsilon}$$
Fill the corresponding function.

*Note:* $\lambda_1$ can be $0$, $\epsilon = 10^{-12}$ is a small constant to prevent undefined values.

In [ ]:
def descriptor_sphericity(eigvals):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvals.shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

d_sphericity = descriptor_sphericity(eigvals)
val_d_sphericity = descriptor_sphericity(val_eigvals)
point_cloud_visu(pts_for_descriptor, d_sphericity[:,0])

**Question:** create the final descriptor by concatenating all the previous descriptors

In [ ]:
## fill the corresponding variables
# descriptor = ...
# val_descriptor = ...

Now that we have the descriptor, we can train the classifier.

**Question:** train a RandomForest ([documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier)) on the training descriptors and predict on the validation descriptors.

In [ ]:
## fill the corresponding variables
# classifier = ...

# point_cloud_visu(val_pts_for_descriptor, color_map[val_predictions])

Visual prediction is good, but not enough.

**Question:** compute the average Intersection over Union (mIoU) and the IoU per class of the predictions. IoU is the [Jaccard score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.jaccard_score.html)

*Note:* do not compute the scores including the class 0, which correspond to the unassigned points.

In [ ]:
## fill the corresponding variables
# print("mIoU", ...)
# print("IoU per class", ...)

## Going further

In the paper [here](https://people.cmm.minesparis.psl.eu/users/marcoteg/cv/publi_pdf/HuguesThomas/2018_3DV_preprint.pdf) several other descriptors derived from the covariance matrix are described and can be implemented.

**Question** Implement the missing descriptors from the paper:
- eigen entropy
- sum of eigen values
- omnivariance
- change of curvature
- absolute moment
- vertical moment
- number of points

In [ ]:
def descriptor_eigen_entropy(eigvals):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvals.shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

def descriptor_sum_of_eigen_values(eigvals):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvals.shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

def descriptor_omnivariance(eigvals):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvals.shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

def descriptor_change_of_curvature(eigvals):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvals.shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

def descriptor_absolute_moment(eigvecs, pts_for_descriptor, pts, indices, order=1):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvecs.shape[0], 1)
  assert(len(desc.shape)==2)
  return desc

def descriptor_vertical_moment(pts_for_descriptor, pts, indices, order=1):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(len(indices).shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

def descriptor_number_of_points(indices):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(len(indices).shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

d_eigen_entropy = descriptor_eigen_entropy(eigvals)
val_d_eigen_entropy = descriptor_eigen_entropy(val_eigvals)
point_cloud_visu(pts_for_descriptor, d_eigen_entropy[:,0])

**Question:** fill the function that compute the complete descriptor

In [ ]:
def compute_descriptor(eigvals, eigvecs, pts_for_descriptor, pts, indices):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(eigvals.shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

print(compute_descriptor(eigvals, eigvecs, pts_for_descriptor, pts, indices).shape)


Another key ingredient is the multiscale descriptor, i.e., concatenating several descriptors for several neighborhood sizes.

**Question:** complete the function that compute the multiscale descriptor.

In [ ]:
def compute_multiscale_descriptor(pts_for_descriptor, pts, radius_list):
  ## fill the function

  # To be removed (only there for compatibility)
  desc = np.zeros(pts_for_descriptor.shape[0], 1)

  assert(len(desc.shape)==2)
  return desc

descriptor = compute_multiscale_descriptor(pts_for_descriptor, pts, [1,2,4])
val_descriptor = compute_multiscale_descriptor(val_pts_for_descriptor, val_pts, [1,2,4])
print(descriptor.shape, cls_for_descriptor.shape)
print(val_descriptor.shape, val_cls_for_descriptor.shape)

**Qestion:** same as before, compute the random forest, the predictions...

In [ ]:
## fill the corresponding variables
# classifier = ...

# point_cloud_visu(val_pts_for_descriptor, color_map[val_predictions])

**Question:** and the metrics

In [ ]:
## fill the corresponding variables
# print("mIoU", ...)
# print("IoU per class", ...)